# Домашнее Задание 2
НИУ ВШЭ, Зарубин Дмитрий

>#### 1. Скачать последовательность генома E.coli.

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
!gunzip GCF_000005845.2_ASM584v2_genomic.fna.gz

--2021-03-11 10:36:27--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.11, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1379902 (1.3M) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’

GCF_000005845.2_ASM 100%[===================>]   1.32M  2.51MB/s    in 0.5s    

2021-03-11 10:36:28 (2.51 MB/s) - ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’ saved [1379902/1379902]



In [ ]:
pip install biopython

     |████████████████████████████████| 2.3MB 10.9MB/s 


In [ ]:
import regex as re
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
from math import log
from statistics import mean
from random import randrange
from Bio import SeqIO

>#### 2. Выделить 1000 фрагментов размером 4000 нуклеотидов. Можно с фиксированным шагом, можно с помощью датчика случайных чисел.

In [ ]:
# Читаем последовательность из файла
genome = str(SeqIO.read('GCF_000005845.2_ASM584v2_genomic.fna','fasta').seq)
print(f'Длина генома | {len(genome)}')

Длина генома | 4641652


In [ ]:
# Делим геном на 1000 последовательностей, каждая длинной 4000 н.
# Случайно бросаем точку в интервале [0; длина генома - 4000]
# Берем фрагмент в интервале [случайная точка; случайная точка+4000]


sequences = [[]for _ in range(1000)]# двойной массив размера (1000, 4000), 
                                    # в котором хранятся все фрагменты.
for i in range(1000):
    r = randrange(len(genome)-4000)
    sequences[i] = genome[r:r+4000]
sequences = np.array(sequences)

>#### 3. Для каждого фрагмента построить бернулиевскую модель и марковские модели 1,2,3 порядка

In [ ]:
# Функция для подсчета частот для модели k-порядка.

def frequences(sequence, k=0):
    bases = ('A', 'C', 'T', 'G')
    freqs = {} # Пустой словарь
    # считаем частоты для каждого k+1-мера последовательности.
    
    for i in range(k, k+2):# perm - пербор всех возможных комбинаций из bases с длиной от k+1 до k+2
        
        freqs[i] = {}
        for perm in list(itertools.product(bases, repeat=i)):
            freqs[i][''.join(perm)] = len(re.findall(''.join(perm), sequence, overlapped=True))/\
                                  (len(sequence) - k)
                                  
    return freqs

In [ ]:
frequences(sequences[0], k=1)

{1: {'A': 0.26206551637909475,
  'C': 0.2528132033008252,
  'G': 0.25406351587896975,
  'T': 0.23130782695673918},
 2: {'AA': 0.09177294323580895,
  'AC': 0.053513378344586145,
  'AG': 0.0622655663915979,
  'AT': 0.05451362840710178,
  'CA': 0.07001750437609403,
  'CC': 0.05901475368842211,
  'CG': 0.07126781695423856,
  'CT': 0.05251312828207052,
  'GA': 0.060515128782195546,
  'GC': 0.08152038009502376,
  'GG': 0.06001500375093773,
  'GT': 0.052013003250812706,
  'TA': 0.03975993998499625,
  'TC': 0.0587646911727932,
  'TG': 0.060515128782195546,
  'TT': 0.07201800450112528}}

In [ ]:
# Функция для подсчета начальных вероятностей - q(марковских моделей)
def starting_freqs(dataframe, k=1):
    bases = ('A', 'C', 'T', 'G')
    freqs = {} # Пустой словарь
    # считаем частоты для префиксов последовательности
    # создаем словарь для хранения частот
    for i in range(1, k+2):# perm - пербор всех возможных комбинаций из bases с длиной от 1 до k+1
        for perm in list(filter(lambda x: len(x) > 0, itertools.product(bases, repeat=i))):
            freqs[''.join(perm)] = 0
        # пробегаем по каждому фрагменту из генома
        # берем отрезок от 0 до k и считаем для него вхождения
        for array in dataframe:
            freqs[array[0:i]] += 1
    # считаем частоты
    for item in freqs:
        freqs[item] = freqs[item]/1000
    return freqs

In [ ]:
starting_freqs(sequences, k=1)

{'A': 0.265,
 'AA': 0.065,
 'AC': 0.067,
 'AG': 0.061,
 'AT': 0.072,
 'C': 0.274,
 'CA': 0.057,
 'CC': 0.072,
 'CG': 0.081,
 'CT': 0.064,
 'G': 0.236,
 'GA': 0.057,
 'GC': 0.085,
 'GG': 0.046,
 'GT': 0.048,
 'T': 0.225,
 'TA': 0.048,
 'TC': 0.042,
 'TG': 0.067,
 'TT': 0.068}

In [ ]:
# Рассчет BIC AIC

def calc_BIC (L,s,n=1000):
    # s - кол-во независимых параметров модели
    return s*log(n) - 2*L


def calc_AIC (L, s):
    return 2*s - 2*L

In [ ]:
# Функция возвращает словарь со скорами AIC и BIC для модели Бернулли.

def Bernoulli (sequence, n=1000, k=0):
    freqs = [[]for _ in range(n)]# Двойной массив размера (1000, 4^k+1) для частот
    BIC = 0# Двойной массив размера (1000, 1) для BIC
    AIC = 0# Двойной массив размера (1000, 1) для AIC
    s = 4**(k + 1) - 1 # Число параметров
    for i in range(n):
        L = 0 # loglikelihood
        freqs[i] = frequences(sequences[i], k=k)[k+1]# для каждого фрагмента считаются частоты (k+1)мера
        for nucleotide in sequences[i]:
            L += log(freqs[i][nucleotide])# бежим по последовательности и 
                                          # прибавляем логарифм часоты для каждого символа. 
        # Считаем AIC BIC для каждого фрагмента.
        AIC += calc_AIC(L, s)
        BIC += calc_BIC(L, s)
    return {'Model' : 'Bernoulli',
            'AIC' : round(AIC),
            'BIC' : round(BIC)}

In [ ]:
# Функция возвращает словарь со скорами AIC и BIC для Марковской модели k-порядка.

def Markov(sequence, n=1000, k=1):
    freqs = [[]for _ in range(n)]# Двойной массив размера (1000, 4^k+1) для частот
    BIC = 0# Двойной массив размера (1000, 4000) для BIC
    AIC = 0# Двойной массив размера (1000, 4000) для AIC
    s = 4**(k + 1) - 1 # Число параметров модели
    q = starting_freqs(sequence, k=k)# Считаем начальные частоты k-меров
    # Для каждой послед-ти
    for i in range(n):
        L = 0
        # прибавляем логарифмы начальных частот 
        for t in range(1,k+1):
            L += log(q[sequence[i][0:t]])
        # Считаем переходные вероятности
        freqs[i] = frequences(sequences[i], k=k)
        for base in list(freqs[i][k+1]):
            # если наткнемся на не встерчающийся k-мер
            if freqs[i][k][base[:-1]] == 0:
                pass
            else:
                freqs[i][k+1][base] = freqs[i][k+1][base]/freqs[i][k][base[:-1]]# переходная вероятность: freqs[AC]/freqs[A]
        # Считаем правдоподобие
        for j in range(k, len(sequence[i])-k):
            L += log(freqs[i][k+1][sequence[i][j:j+k+1]])
        AIC += calc_AIC(L, s)
        BIC += calc_BIC(L, s)
    return {'Model' : f'Markov_{k}',
            'AIC' : round(AIC),
            'BIC' : round(BIC)}

>#### 4. Для каждой модели оценить AIC и BIC.

In [ ]:
final = [Bernoulli(sequences),]

In [ ]:
for k in tqdm(range(1,5)):
    final.append(Markov(sequences,k=k))


100%|██████████| 4/4 [02:58<00:00, 44.55s/it]


In [ ]:
df = pd.DataFrame(final, index = [i for i in range(1, 6)])
df

,Model,AIC,BIC
1,Bernoulli,11064948,11079671
2,Markov_1,10958569,11032185
3,Markov_2,10892374,11201563
4,Markov_3,11041494,12292972
5,Markov_4,11864421,16885055


>#### 5. Сделать вывод – какая модель лучше описывает последовательности.

In [ ]:
min_AIC = df['AIC'].idxmin()
min_BIC = df['BIC'].idxmin()
print(f'Best model by AIC | {df.Model[min_AIC]}\n\nBest model by BIC | {df.Model[min_BIC]}')

Best model by AIC | Markov_2

Best model by BIC | Markov_1


>Если судить по критерию AIC лучше всего фрагменты генома *E.coli* описывает марковская модель 2 порядка.<br/>
Если же смотреть по критерию BIC, то лучшая - марковская модель 1 порядка.
Исходя из этого, наверное, можно судить о "триплетности" генома